<a href="https://colab.research.google.com/github/405620294/iShape/blob/main/train_waist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect your google drive，Please follow the steps
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# before run this ipynb, u need to notice that:
# 1. 欄位順序如下 ：['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']
# 2. 請先 在KNN區塊，　定義要使用的　scaler，　並且 線性回歸都是 使用相同的 scaler
# 3. 在預測資料前，請先定義 model_name = 要用的演算法

# 水下秤重 dataset >> 預測腰圍


## raw ( original dataset ) >> df ( preprocessed dataset ) >> df_input ( filtered column dataframe ) >> slicing target ( x_df / y_df )

In [ ]:
import pandas as pd
bw_df =  pd.read_csv("/content/drive/Shareddrives/AI_Python/ishape/ai/step1/hps_output/body_width.csv", encoding="utf-8")

# 資料預處理

In [ ]:
### 預處理 ###

bw_df = bw_df[['org_img', 'age', 'gender', 'race', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'chest', 'waist', 'hips', 'bfp', 'source']]

# 缺失值 > 全刪
bw_df.dropna(axis=0, how='any', inplace=True)

# 性別欄位 m:0 f:1 >> 二質型資料可以不做one-hot-encoding
dic = { 'm' : 0, 'f' : 1}
bw_df['gender'] = bw_df['gender'].replace(dic)

# 種族欄位 e:0 w:1 >> 二質型資料可以不做one-hot-encoding
dic = { 'e' : 0, 'w' : 1}
bw_df['race'] = bw_df['race'].replace(dic)

# 身高轉公尺平方
def t_m2(t):
  t = t / 100
  return t * t
m2 = bw_df['height'].apply(t_m2)

# 新增欄位 bmi
bw_df['bmi'] = bw_df['weight'] / m2

print('bw_df 共', len(bw_df), '筆資料')

bw_df 共 262 筆資料


# 資料篩選

In [ ]:
### 條件篩選 (挑選進入訓練的資料) ###
df = bw_df

# 篩選性別欄位 m:0 f:1
# delete_idx = df.loc[df["gender"] == 0].index.tolist() # 去掉以下欄位
# df = df.drop(index=delete_idx)

# 篩選種族欄位 e:0 w:1
# delete_idx = df.loc[df["race"] == 1].index.tolist() # 去掉以下欄位
# df = df.drop(index=delete_idx)

# 篩選source欄位
c = '水下秤重'
delete_idx = df.loc[df["source"] != c].index.tolist() 
df = df.drop(index=delete_idx)

# bfp < 5 刪除
delete_idx = df.loc[df['bfp'] < 5].index.tolist()
df = df.drop(index=delete_idx)

print('df 共', len(df), '筆資料')

df 共 246 筆資料


# 欄位篩選

In [ ]:
### 欄位刪除 (刪除不用進入訓練欄位) ###

# 刪除體脂率、org_img、gender、race、source
df_input = df.drop(['bfp', 'org_img', 'gender', 'race', 'source'], axis=1)

print('完成資料與欄位挑選，將進入訓練的資料， df_input 共', len(df_input), '筆')
print('df_input 欄位 :', df_input.columns)

完成資料與欄位挑選，將進入訓練的資料， df_input 共 246 筆
df_input 欄位 : Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'chest', 'waist', 'hips', 'bmi'],
      dtype='object')


# 準備訓練資料 & 資料切片

In [ ]:
x_df = df_input[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']] # 欄位排序
y_df = df_input['waist']

In [ ]:
print(' 訓練資料 x_df 共', len(x_df), ' ,欄位 : ', x_df.columns)
print(' 訓練資料 y_df 共', len(y_df), ' ,欄位 : ', y_df.name)

 訓練資料 x_df 共 246  ,欄位 :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 訓練資料 y_df 共 246  ,欄位 :  waist


In [ ]:
# 資料切片
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.array(x_df), np.array(y_df), test_size=0.1)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score

# Regression Tree

In [ ]:
### dt ###

from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
params = {"max_depth":range(3, 10)} # the search range of Grid Search
grid = GridSearchCV(dt, params, cv=10, n_jobs=-1, scoring="neg_root_mean_squared_error")
grid.fit(x_df, y_df)
print('the best max depth of DecisionTreeRegressor : ', grid.best_params_)

dt = DecisionTreeRegressor(max_depth = grid.best_params_['max_depth'])
dt.fit(x_train, y_train)
pre = dt.predict(x_test)
print('dt_r2: ', r2_score(y_test, pre))
print('dt_MAE: ', mean_absolute_error(y_test, pre))
print('dt_MSE: ', mean_squared_error(y_test, pre))

# Evaluate a score by cross-validation
dt_cross_r2 = cross_val_score(dt, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
dt_cross_RMSE = cross_val_score(dt, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

the best max depth of DecisionTreeRegressor :  {'max_depth': 6}
dt_r2:  0.787250301055371
dt_MAE:  3.1560396825396815
dt_MSE:  15.87026292649281


# Random Forests Regression

In [ ]:
### rf ###

from sklearn.ensemble import RandomForestRegressor

# find the best estimators & max depth of rf
rf = RandomForestRegressor()
params = {
    "n_estimators":range(10, 101, 2),
    "max_depth":range(3, 10)
}
grid = GridSearchCV(rf, params, cv=10, n_jobs=-1, scoring="neg_root_mean_squared_error")
grid.fit(x_df, y_df)
print('best_score : ', grid.best_score_, 'the best params of DecisionTreeRegressor : ', grid.best_params_)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_depth = grid.best_params_['max_depth'])
rf.fit(x_df, y_df)

pre = rf.predict(x_test)
print('rf_r2: ', r2_score(y_test, pre))
print('rf_MAE: ', mean_absolute_error(y_test, pre))
print('rf_MSE: ', mean_squared_error(y_test, pre))

# Evaluate a score by cross-validation
rf_cross_r2 = cross_val_score(rf, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
rf_cross_RMSE = cross_val_score(rf, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

best_score :  -3.0897135043871384 the best params of DecisionTreeRegressor :  {'max_depth': 9, 'n_estimators': 52}
rf_r2:  0.9839401046643614
rf_MAE:  0.8761202435963434
rf_MSE:  1.1980029246239896


In [ ]:
#找出 隨機森林特徵的重要性
pd.DataFrame(rf.feature_importances_, 
       columns=["Importance"],
       index=x_df.keys()).sort_values(by="Importance", ascending=False)
       #ascending=False降冪排列

,Importance
waist_width_cm,0.686336
bmi,0.230770
weight,0.034839
chest_width_cm,0.022220
age,0.010472
hip_width_cm,0.009359
height,0.006004


# SVR Regression

In [ ]:
### svr回歸 ###
from sklearn.svm import SVR
linear_svr = SVR(kernel='linear') # 使用线性核函数配置
poly_svr = SVR(kernel='poly') # 使用多项式核函数配置
rbf_svr = SVR(kernel='rbf', C=100)

linear_svr.fit(x_train, y_train)
poly_svr.fit(x_train, y_train)
rbf_svr.fit(x_train, y_train)

# Evaluate a score by cross-validation
linear_svr_r2 = cross_val_score(linear_svr, np.array(x_df), np.array(y_df), cv=10, scoring="r2") 
linear_svr_RMSE = cross_val_score(linear_svr, np.array(x_df), np.array(y_df), cv=10, scoring="neg_root_mean_squared_error")

poly_svr_r2 = cross_val_score(poly_svr, np.array(x_df), np.array(y_df), cv=10, scoring="r2") 
poly_svr_RMSE = cross_val_score(poly_svr, np.array(x_df), np.array(y_df), cv=10, scoring="neg_root_mean_squared_error") 

rbf_svr_r2 = cross_val_score(rbf_svr, np.array(x_df), np.array(y_df), cv=10, scoring="r2") 
rbf_svr_RMSE = cross_val_score(rbf_svr, np.array(x_df), np.array(y_df), cv=10, scoring="neg_root_mean_squared_error")

# KNN Regression

In [ ]:
# prepare another dataset for KNN & linear ( cause data needs to Normalization before training) 
from sklearn.preprocessing import RobustScaler, MaxAbsScaler, MinMaxScaler, StandardScaler
def do_scaler(name, input_df):
  scaler = name()
  scaler.fit(input_df)
  return scaler

# choose a Normalization which u want to do by yourself !
scaler = MinMaxScaler # u can choose : RobustScaler, MaxAbsScaler, MinMaxScaler, StandardScaler

df_need_scaler = df_input.drop(['chest', 'waist', 'hips'], axis=1)
knn_scaler = do_scaler(scaler, df_need_scaler) # fit the scaler
scalered_df = pd.DataFrame(knn_scaler.transform(df_need_scaler), columns = df_need_scaler.columns) # transform the dataframe
print('您已對', df_need_scaler.columns , '欄位，使用　', scaler, '縮放資料')

您已對 Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object') 欄位，使用　 <class 'sklearn.preprocessing._data.MinMaxScaler'> 縮放資料


In [ ]:
x_df = scalered_df[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']] # 欄位排序
y_df = df_input['waist']

print(' 訓練資料 x_df 共', len(x_df), ' ,欄位 : ', x_df.columns)
print(' 訓練資料 y_df 共', len(y_df), ' ,欄位 : ', y_df.name)

 訓練資料 x_df 共 246  ,欄位 :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 訓練資料 y_df 共 246  ,欄位 :  waist


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x_df), np.array(y_df), test_size=0.1)

In [ ]:
### knn ###
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
params = {"n_neighbors":range(1, 50)}
grid = GridSearchCV(knn, params, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')
grid.fit(x_df, y_df)

knn = KNeighborsRegressor(n_neighbors = grid.best_params_['n_neighbors'])
knn.fit(x_train, y_train)
pre = knn.predict(x_test)
print('knn_r2: ', r2_score(y_test, pre))
print('Knn_MAE: ', mean_absolute_error(y_test, pre))
print('Knn_MSE: ', mean_squared_error(y_test, pre))

# Evaluate a score by cross-validation
knn_cross_r2 = cross_val_score(knn, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
knn_cross_RMSE = cross_val_score(knn, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

knn_r2:  0.8484158615560011
Knn_MAE:  3.3748571428571434
Knn_MSE:  16.59991836734693


# Linear Regression

In [ ]:
# prepare another dataset cause datas' distribution needs to be normal before trainning any Linear Regression)

# find the skewness of each columns
from scipy.stats import skew
def getskew(series):
  return skew(series.dropna() + 1)

# adjust skewed to normal
from scipy.stats import boxcox_normmax
def get_lamda(s):
  lamda = boxcox_normmax(s.dropna() + 1)
  return lamda

from scipy.special import boxcox1p, inv_boxcox1p
def sk_normal(s):
  lamda = boxcox_normmax(s.dropna() + 1)
  return boxcox1p(s, lamda)


# 查看欄位偏度
column = df_input.columns # 所有df_input名稱
skewness = (df_input[column].apply(getskew).sort_values(ascending=False))
print(skewness)

# 拿到各欄位lamda & let series change to dictionary
lamdas = df_input[column].apply(get_lamda)
lamdas = lamdas.to_dict()

# 調整後偏度
df_input[column] = df_input[column].apply(sk_normal)
skewness = (df_input[column].apply(getskew).sort_values(ascending=False))
print('修正後偏度')
print(skewness)

bmi               0.696487
hip_width_cm      0.548198
hips              0.503970
chest_width_cm    0.473728
chest             0.464332
waist             0.410822
weight            0.366416
waist_width_cm    0.331624
age               0.259460
height            0.123417
dtype: float64
修正後偏度
hip_width_cm      0.028584
age               0.022386
waist_width_cm    0.011417
chest             0.003573
waist             0.001612
weight           -0.001218
height           -0.001631
bmi              -0.007760
chest_width_cm   -0.012893
hips             -0.012945
dtype: float64


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3538: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [ ]:
# prepare target variable and Scaler input variable

df_need_scaler = df_input.drop(['chest', 'waist', 'hips'], axis=1)
# you already choosed the Normalization at the KNN Regression block
linear_scaler = do_scaler(scaler, df_need_scaler) # fit the scaler
scalered_df = pd.DataFrame(linear_scaler.fit_transform(df_need_scaler), columns = df_need_scaler.columns) # transform t
print('您已對', df_need_scaler.columns , '欄位，使用　', scaler, '縮放資料')

您已對 Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object') 欄位，使用　 <class 'sklearn.preprocessing._data.MinMaxScaler'> 縮放資料


In [ ]:
x_df = scalered_df[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']] # 欄位排序
y_df = df_input['waist']

print(' 訓練資料 x_df 共', len(x_df), ' ,欄位 : ', x_df.columns)
print(' 訓練資料 y_df 共', len(y_df), ' ,欄位 : ', y_df.name)

 訓練資料 x_df 共 246  ,欄位 :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 訓練資料 y_df 共 246  ,欄位 :  waist


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x_df), np.array(y_df), test_size=0.1)

In [ ]:
# linear ###
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(x_train, y_train)

pre = linear.predict(x_test)
print('linear_r2  ', r2_score(y_test, pre))
print('linear_MAE 　', mean_absolute_error(y_test, pre))
print('linear_MSE  ', mean_squared_error(y_test, pre))

linear_cross_r2 = cross_val_score(linear, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
linear_cross_RMSE = cross_val_score(linear, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

linear_r2   0.9378462975055902
linear_MAE 　 0.002977756023245517
linear_MSE   1.4372600200575408e-05


In [ ]:
coeffecients = pd.DataFrame(linear.coef_,x_df.columns)
coeffecients.columns = ['Coeffecient'] 
coeffecients

,Coeffecient
age,0.004495
height,-0.015240
weight,0.039566
chest_width_cm,-0.046453
waist_width_cm,0.092031
hip_width_cm,0.001449
bmi,-0.012979


# Lasso Regression

In [ ]:
### lasso ### 
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet

cv = LassoCV(cv=10)
cv.fit(x_train, y_train)
print('LassoCV', cv.alpha_)

lasso = Lasso(cv.alpha_)
lasso.fit(x_train, y_train)
pre = lasso.predict(x_test)

print('lasso_r2  ', r2_score(y_test, pre))
print('lasso_MAE 　', mean_absolute_error(y_test, pre))
print('lasso_MSE  ', mean_squared_error(y_test, pre))

lasso_cross_r2 = cross_val_score(lasso, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
lasso_cross_RMSE = cross_val_score(lasso, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

LassoCV 2.5546976704266464e-06
lasso_r2   0.9352258782135122
lasso_MAE 　 0.003072397523753132
lasso_MSE   1.4978553463718491e-05


# Ridge Regression

In [ ]:
### ridge ###
cv = RidgeCV(cv=10)
cv.fit(x_train, y_train)
print(cv.alpha_)

ridge = Ridge(cv.alpha_)
ridge.fit(x_train, y_train)
pre = ridge.predict(x_test)

print('ridge_r2  ', r2_score(y_test, pre))
print('ridge_MAE 　', mean_absolute_error(y_test, pre))
print('ridge_MSE  ', mean_squared_error(y_test, pre))

ridge_cross_r2 = cross_val_score(ridge, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
ridge_cross_RMSE = cross_val_score(ridge, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

0.1
ridge_r2   0.9222226867939622
ridge_MAE 　 0.0033381086782717072
ridge_MSE   1.7985448694482165e-05


In [ ]:
coeffecients = pd.DataFrame(ridge.coef_,x_df.columns) 
coeffecients.columns = ['Coeffecient'] 
coeffecients

,Coeffecient
age,0.006087
height,-0.004031
weight,0.020550
chest_width_cm,-0.031911
waist_width_cm,0.066744
hip_width_cm,0.004741
bmi,0.015656


# Elastic Net Regression

In [ ]:
### ElasticNet ###
cv = ElasticNetCV(cv=10)
cv.fit(x_train, y_train)
print(cv.alpha_)

elastic = ElasticNet(cv.alpha_)
elastic.fit(x_train, y_train)
pre = elastic.predict(x_test)

print('Eelastic_r2  ', r2_score(y_test, pre))
print('elastic_MAE 　', mean_absolute_error(y_test, pre))
print('elastic_MSE  ', mean_squared_error(y_test, pre))

elastic_cross_r2 = cross_val_score(elastic, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
elastic_cross_RMSE = cross_val_score(elastic, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

5.1093953408532936e-06
Eelastic_r2   0.9351617279028904
elastic_MAE 　 0.0030736973640705044
elastic_MSE   1.4993387765301607e-05


# 交叉驗證訓練分數

In [ ]:
# cross-validation ( CV = 10 ) 

print('dt_cross_r2: ', np.average(dt_cross_r2))
print('dt_cross_RMSE ', np.average(dt_cross_RMSE))
print('-------------------------------------------')

print('rf_cross_r2: ', np.average(rf_cross_r2))
print('rf_cross_RMSE', np.average(rf_cross_RMSE))
print('-------------------------------------------')

print('linear_svr_r2 ', np.average(linear_svr_r2), ' linear_svr_RMSE ', np.average(linear_svr_RMSE))
print('poly_svr_r2   ', np.average(poly_svr_r2), '  poly_svr_RMSE ', np.average(poly_svr_RMSE))
print('rbf_svr_r2    ', np.average(rbf_svr_r2), '   rbf_svr_RMSE ', np.average(rbf_svr_RMSE))
print('-------------------------------------------')

print('knn_cross_r2: ', np.average(knn_cross_r2))
print('knn_cross_RMSE ', np.average(knn_cross_RMSE))
print('-------------------------------------------')

print('linear_cross_r2  ', np.average(linear_cross_r2))
print('linear_cross_RMSE  ', np.average(lasso_cross_RMSE))
print('-------------------------------------------')

print('lasso_cross_r2  ', np.average(lasso_cross_r2))
print('lasso_cross_RMSE  ', np.average(lasso_cross_RMSE))
print('-------------------------------------------')

print('ridge_cross_r2  ', np.average(ridge_cross_r2))
print('ridge_cross_RMSE  ', np.average(ridge_cross_RMSE))
print('-------------------------------------------')

print('elastic_cross_r2  ', np.average(elastic_cross_r2))
print('elastic_cross_RMSE  ', np.average(elastic_cross_RMSE))


print('-------------------------------------------')
print('linear, Lasso, Ridge and ElasticNet had adjusted skewed to normal')
print('knn, linear, Lasso, Ridge and ElasticNet had adjusted the range with' , scaler)
print(' 訓練資料 x_df 共', len(x_df), '筆資料 ,欄位 : ', x_df.columns)
print(' 訓練資料 y_df 共', len(y_df), ' ,欄位 : ', y_df.name)

dt_cross_r2:  0.7566295116145301
dt_cross_RMSE  -3.7094196693232617
-------------------------------------------
rf_cross_r2:  0.8333398260207578
rf_cross_RMSE -3.1804108393690678
-------------------------------------------
linear_svr_r2  0.902085090927519  linear_svr_RMSE  -2.536861696600336
poly_svr_r2    0.7884319167368733   poly_svr_RMSE  -3.737225491965875
rbf_svr_r2     0.8481899929262063    rbf_svr_RMSE  -3.063886973388292
-------------------------------------------
knn_cross_r2:  0.7800469754713542
knn_cross_RMSE  -3.9120432362382664
-------------------------------------------
linear_cross_r2   0.9059803725470605
linear_cross_RMSE   -0.0037335223006309468
-------------------------------------------
lasso_cross_r2   0.9054625086897719
lasso_cross_RMSE   -0.0037335223006309468
-------------------------------------------
ridge_cross_r2   0.8979615558272978
ridge_cross_RMSE   -0.003872758183215729
-------------------------------------------
elastic_cross_r2   0.905454948425126
elast

# 預測資料

In [ ]:
### please adjust 'model_name' by yourself ! ###

# u can choose : dt, rf, linear_svr, poly_svr, rbf_svr, knn, linear, lasso, ridge, elastic
model_name = rf 
algorithm = str(model_name).split('(')[0]

In [ ]:
### 條件篩選 (挑選進入預測的資料) ###
df = bw_df

# 篩選性別欄位 m:0 f:1
delete_idx = df.loc[df["gender"] == 1].index.tolist() # 去掉以下欄位
df = df.drop(index=delete_idx)

# 篩選種族欄位 e:0 w:1
# delete_idx = df.loc[df["race"] == 1].index.tolist() # 去掉以下欄位
# df = df.drop(index=delete_idx)

# 篩選source欄位
# c = '水下秤重'
# delete_idx = df.loc[df["source"] != c].index.tolist() 
# df = df.drop(index=delete_idx)

# bfp < 5 刪除
delete_idx = df.loc[df['bfp'] < 5].index.tolist()
df = df.drop(index=delete_idx)

print('df 共', len(df), '筆資料')

df 共 256 筆資料


In [ ]:
### 欄位刪除 (挑選進入預測的欄位) ###

# 刪除體脂率、org_img、gender、race、source
predict_df = df[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']]

print('完成資料與欄位挑選，將進入訓練的資料， predict_df 共', len(predict_df), '筆')
print('predict_df 欄位 :', predict_df.columns)

完成資料與欄位挑選，將進入訓練的資料， predict_df 共 256 筆
predict_df 欄位 : Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')


In [ ]:
###　adjust skewed to normal ###

need_adjust_skewed = ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

if algorithm in need_adjust_skewed :
  for i in predict_df.keys():
    predict_df[i]= boxcox1p(predict_df[i], lamdas[i])
  print('現在預測資料的演算法 :', algorithm, ', predict_df 已完成偏度調整')

else:
  print('現在預測資料的演算法 :', algorithm, ', predict_df 不須做偏度調整')

現在預測資料的演算法 : RandomForestRegressor , predict_df 不須做偏度調整


In [ ]:
###　scaler ###
need_scaler = ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

if algorithm == 'KNeighborsRegressor':
  predict_df = pd.DataFrame(knn_scaler.transform(predict_df), columns = predict_df.columns)
  print('現在預測資料的演算法 :', algorithm, ' predict_df 已完成', scaler)

elif algorithm in need_scaler:
  predict_df = pd.DataFrame(linear_scaler.transform(predict_df), columns = predict_df.columns)
  print('現在預測資料的演算法 :', algorithm, ' predict_df 已完成', scaler)

else:
  print('現在預測資料的演算法 :', algorithm, ' predict_df 不須做任何scaler')

現在預測資料的演算法 : RandomForestRegressor  predict_df 不須做任何scaler


In [ ]:
### 進入模型預測體脂率 pre ###

pre = model_name.predict(predict_df).round(4)

need_inv_skewed = ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

if algorithm in need_inv_skewed:
  pre = inv_boxcox1p(pre, lamdas['waist'])
  print('現在預測資料的演算法 :', algorithm, ' 預測值已完成偏度還原',)
else:
  print('現在預測資料的演算法 :', algorithm, ' 預測值不須調整偏度')

現在預測資料的演算法 : RandomForestRegressor  預測值不須調整偏度


In [ ]:
### 查看預測結果 & 輸出 output dataframe ###

output = df
# 調整waist欄位順序
number_col = output.pop(df.columns[10])
output.insert(14, number_col.name, number_col)

output['pre_waist'] = pre
output['pre_waist - real_waist'] = (output['pre_waist'] - output['waist']).abs()

# 性別欄位 m:0 f:1 
dic = { 0 : '男性', 1 : '女性'}
output['gender'] = output['gender'].replace(dic)

# 種族欄位 e:0 w:1 
dic = { 0 : '東方人', 1 : '西方人'}
output['race'] = output['race'].replace(dic)

print('現在預測資料的演算法 :　', algorithm, ' ,使用的scaler :　', scaler)
print('完成預測， output 共', len(output), '筆')

現在預測資料的演算法 :　 RandomForestRegressor  ,使用的scaler :　 <class 'sklearn.preprocessing._data.MinMaxScaler'>
完成預測， output 共 256 筆


In [ ]:
### 體脂肪誤差 > 5 ###
print('體脂肪誤差 > 5 共', len(output[output['pre_waist - real_waist'] >= 5]), '筆資料')
output[output['pre_waist - real_waist'] >= 5].sort_values(['pre_waist - real_waist'], ascending=False)

體脂肪誤差 > 5 共 6 筆資料


,org_img,age,gender,race,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,chest,hips,bfp,source,bmi,waist,pre_waist,pre_waist - real_waist
419,8704.jpg,24.0,男性,東方人,161.5,50.5,36.355,33.512,36.056,77.0,88.0,9.2,真人,19.361827,66.5,85.9736,19.4736
406,8691.jpg,26.0,男性,東方人,176.0,59.0,35.278,33.522,36.874,83.0,91.0,14.3,真人,19.047004,74.0,86.0841,12.0841
417,8702.jpg,24.0,男性,東方人,168.0,92.7,38.232,34.934,40.332,99.5,110.0,32.0,真人,32.844388,89.0,97.8777,8.8777
418,8703.jpg,29.0,男性,東方人,182.0,72.0,35.490,35.976,38.407,88.0,96.5,17.2,真人,21.736505,83.0,91.4338,8.4338
410,8695.jpg,29.0,男性,東方人,172.0,72.9,36.602,34.125,34.125,95.0,99.0,21.9,真人,24.641698,87.0,92.5857,5.5857
405,8690.jpg,24.0,男性,東方人,165.0,48.1,31.133,27.241,35.997,76.5,84.0,11.9,真人,17.667585,68.0,73.2654,5.2654


In [ ]:
### 真人的誤差 ###
output[output['source'] == '真人'].sort_values(['pre_waist - real_waist'], ascending=False)

,org_img,age,gender,race,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,chest,hips,bfp,source,bmi,waist,pre_waist,pre_waist - real_waist
419,8704.jpg,24.0,男性,東方人,161.5,50.5,36.355,33.512,36.056,77.0,88.0,9.2,真人,19.361827,66.5,85.9736,19.4736
406,8691.jpg,26.0,男性,東方人,176.0,59.0,35.278,33.522,36.874,83.0,91.0,14.3,真人,19.047004,74.0,86.0841,12.0841
417,8702.jpg,24.0,男性,東方人,168.0,92.7,38.232,34.934,40.332,99.5,110.0,32.0,真人,32.844388,89.0,97.8777,8.8777
418,8703.jpg,29.0,男性,東方人,182.0,72.0,35.490,35.976,38.407,88.0,96.5,17.2,真人,21.736505,83.0,91.4338,8.4338
410,8695.jpg,29.0,男性,東方人,172.0,72.9,36.602,34.125,34.125,95.0,99.0,21.9,真人,24.641698,87.0,92.5857,5.5857
405,8690.jpg,24.0,男性,東方人,165.0,48.1,31.133,27.241,35.997,76.5,84.0,11.9,真人,17.667585,68.0,73.2654,5.2654
414,8699.jpg,28.0,男性,東方人,168.0,67.6,37.740,32.049,37.440,90.0,95.0,21.2,真人,23.951247,81.0,85.5387,4.5387
404,8689.jpg,29.0,男性,東方人,166.0,64.8,34.913,31.118,39.011,87.0,95.0,22.6,真人,23.515750,82.0,85.0447,3.0447
407,8692.jpg,23.0,男性,東方人,179.0,98.7,36.892,36.892,42.291,103.0,114.0,26.1,真人,30.804282,105.0,102.7042,2.2958
408,8693.jpg,29.0,男性,東方人,174.0,73.0,34.555,31.614,38.819,91.0,99.0,21.1,真人,24.111507,87.0,87.5460,0.5460


# 存模型 >> 預測體脂肪
模型與預測結果，直接倒進google drive

In [ ]:
# import time
# from sklearn.externals import joblib
# import datetime
# import os
# import json

# now = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S') # 當前時間(注意為伺服器時間)
# print('當前時間', now)

# folder = algorithm # 以演算法區分資料夾
# drive_path = os.path.join('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/waist', folder)

# if not os.path.exists(drive_path):
#   os.makedirs(drive_path)

# # 存模型
# name_model = folder + '_pre_waist_' + str(now) + '.h5'
# joblib.dump(model_name, os.path.join(drive_path, name_model))

# if algorithm in ['KNeighborsRegressor', 'LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']:

#   if algorithm == 'KNeighborsRegressor':
#     save_scaler = knn_scaler
#   elif algorithm in ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']:
#     save_scaler = linear_scaler

#     # 存skewness
#     name_lamda = folder + '_lamda_' + str(now) + '.json'
#     f = open(os.path.join(drive_path, name_lamda), "w", encoding="UTF-8") #寫入
#     json.dump(lamdas , f, ensure_ascii=False, indent=4) #輸出成json檔案
#     f.close()
#     print('已儲存lamdas 參數')


#   # 存scaler
#   name_scaler = folder + '_' + str(save_scaler).split('(')[0] + '_' + str(now) + '.h5'
#   joblib.dump(save_scaler, os.path.join(drive_path, name_scaler))
#   print('已儲存scaler model')

# 倒出預測結果

In [ ]:
# ### 將預測結果倒出EXCEL 與模型位置同 ###
# import os

# table_name = name_model.split('.')[0] + '.xlsx' # Ridge_pre_bodyfat_2021-05-13_10:34:31.xlsx
# writer = pd.ExcelWriter(os.path.join(drive_path, table_name), engine='openpyxl')
# sheets =  y_df.name + '_' + str(len(df_input)) +  '_' + algorithm
# output.to_excel(writer, sheet_name = sheets, index=False, encoding='utf-8')
# writer.save()